# Build and upload index

In [ ]:
import dotenv
dotenv.load_dotenv()
"HF_TOKEN" in os.environ or print("Please set the HF_TOKEN environment variable in .env file")

True

In [27]:
from evidence_seeker.retrieval import build_index, RetrievalConfig
import os


In [ ]:
config = RetrievalConfig(
    embed_backend_type="huggingface",
    embed_model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    document_input_dir="../TMP/APUZ/corpus",
    index_persist_path="../TMP/APUZ/storage/index",
)

In [41]:
from loguru import logger
import pathlib
import yaml
from llama_index.core import SimpleDirectoryReader
from pprint import pprint

metadata_dict = {}
metadata_dir = config.document_input_dir or os.path.dirname(config.document_input_files[0])
# load and parse all yaml files in metadata_dir
for filepath in pathlib.Path(metadata_dir).rglob("*.yaml"):
    with open(filepath, "r") as f:
        data = yaml.safe_load(f)
        if "file" in data and "author" and "url" in data:
            filename = data["file"]
            author = data["author"]
            url = data["url"]
            metadata_dict[filename] = {"author": author, "url": url}
        else:
            logger.warning(f"Invalid metadata in {filepath}.")
document_file_metadata = lambda filename: metadata_dict.get(pathlib.Path(filename).name, {})
reader = SimpleDirectoryReader(
        input_dir=metadata_dir,
        file_metadata=document_file_metadata,
)
docs = reader.load_data()
pprint(docs[0].metadata)


{'author': 'Ahrens, Ralf',
 'file_name': 'ahrens_pragmatischer_2024.pdf',
 'page_label': '1',
 'url': 'https://www.bpb.de/shop/zeitschriften/apuz/industriepolitik-2024/544578/pragmatischer-interventionismus/'}


In [ ]:

build_index(
    document_input_dir=config.document_input_dir,
    document_file_metadata=document_file_metadata,
    index_persist_path=config.index_persist_path,
    embed_base_url=config.embed_base_url,
    embed_model_name=config.embed_model_name,
    embed_backend_type=config.embed_backend_type,
)

In [7]:
import huggingface_hub

HfApi = huggingface_hub.HfApi(token=os.environ["HF_TOKEN"])

In [8]:
HfApi.upload_folder(
    repo_id="DebateLabKIT/apuz-index-es",
    folder_path=config.index_persist_path,
    path_in_repo="index",
    repo_type="dataset",
)

default__vector_store.json:   0%|          | 0.00/196M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

docstore.json:   0%|          | 0.00/88.5M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/DebateLabKIT/apuz-index-es/commit/6dbdaf9bb91058ed8419731cb8439766c941c16f', commit_message='Upload folder using huggingface_hub', commit_description='', oid='6dbdaf9bb91058ed8419731cb8439766c941c16f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/DebateLabKIT/apuz-index-es', endpoint='https://huggingface.co', repo_type='dataset', repo_id='DebateLabKIT/apuz-index-es'), pr_revision=None, pr_num=None)